In [4]:
import time
import gym
import numpy as np
from gym import spaces
from tqdm import tqdm


import numpy as np
from utils.skeleton import *
from utils.quaternion import *
from utils.blazepose import blazepose_skeletons
import os
from pypot.creatures import PoppyTorso
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#from pypot.creatures.ik import IKChain
from pypot.primitive.move import Move
from pypot.primitive.move import MovePlayer

import sys
sys.path.append('E:\Anaconda\envs\gym-examples')
import gym_examples
from gym.wrappers import FlattenObservation

In [5]:
env = gym.make('gym_examples/Poppy-v0')
env.reset()

pygame 2.1.0 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Hello, I am Poppy!


array([ 0.10876056, -0.17632997,  0.07149935, -0.10220377, -0.18020962,
        0.07149935], dtype=float32)

In [ ]:
# from stable_baselines3.common.env_checker import check_env
# check_env(env)

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

#env = gym.make("Pendulum-v1")

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=100, log_interval=10)
model.save("ddpg_imitation")
vec_env = model.get_env()

del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_imitation")

obs = vec_env.reset()

for t in env.targets:
    action, _states = model.predict(np.array(t.flatten()).reshape(1,-1))
    obs, rewards, dones, info = vec_env.step(action)

In [ ]:
obs = vec_env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = vec_env.step(action)

for t in env.targets:
    action, _states = model.predict(np.array(t.flatten()).reshape(1,-1))
    obs, rewards, dones, info = vec_env.step(action)

In [7]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

#env = gym.make("Pendulum-v1")

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)

model = DDPG.load("ddpg_imitation")
model.get_env()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:

for t in env.targets:
    action, _states = model.predict(np.array(t.flatten()).reshape(1,-1))
    obs, rewards, dones, info = env.step(action.flatten())

current step :  0
reward :  -0.4454171387169538
current step :  1
reward :  -0.44473869396998483
current step :  2
reward :  -0.4451434100122294
current step :  3
reward :  -0.44568206695897744
current step :  4
reward :  -0.4450343231725972
current step :  5
reward :  -0.44417112357213995
current step :  6
reward :  -0.443009084795555
current step :  7
reward :  -0.4415157493820922
current step :  8
reward :  -0.4398868812963723
current step :  9
reward :  -0.4357594938479735
current step :  10
reward :  -0.43171974228216603
current step :  11
reward :  -0.42777305854163317
current step :  12
reward :  -0.42311643325188036
current step :  13
reward :  -0.4186179169447009
current step :  14
reward :  -0.41442517075468244
current step :  15
reward :  -0.41019153841559336
current step :  16
reward :  -0.40547718810164
current step :  17
reward :  -0.4008322441878091
current step :  18
reward :  -0.3970783469804646
current step :  19
reward :  -0.3946795807726316
current step :  20
reward

current step :  165
reward :  -0.5440039658113286
current step :  166
reward :  -0.5477939260501248
current step :  167
reward :  -0.550086575382025
current step :  168
reward :  -0.5533433223663353
current step :  169
reward :  -0.5549007572283674
current step :  170
reward :  -0.5548297178454745
current step :  171
reward :  -0.5545580126864844
current step :  172
reward :  -0.5554400523560253
current step :  173
reward :  -0.5571349463121086
current step :  174
reward :  -0.5587942653122905
current step :  175
reward :  -0.560640217040356
current step :  176
reward :  -0.5628135213868934
current step :  177
reward :  -0.5640167254938844
current step :  178
reward :  -0.5648649753926208
current step :  179
reward :  -0.5653777656268492
current step :  180
reward :  -0.5655619687397987
current step :  181
reward :  -0.5656809780988776
current step :  182
reward :  -0.5661595429426984
current step :  183
reward :  -0.5664330871624297
current step :  184
reward :  -0.5669773953739655
cu

In [16]:
action.flatten()

array([-166.87225  ,  -50.67862  , -155.58098  ,   74.15871  ,
         57.36116  ,  143.42847  ,   -3.2159424, -122.096664 ,
         51.43799  ,   28.526123 ,  153.29846  ,  -71.49954  ,
        -80.46852  ,  -32.0878   ], dtype=float32)